In [ ]:
############################################
# CUSTOMER LIFETIME VALUE (Müşteri Yaşam Boyu Değeri)
############################################

# 1. Veri Hazırlama
# 2. Average Order Value (average_order_value = total_price / total_transaction)
# 3. Purchase Frequency (total_transaction / total_number_of_customers)
# 4. Repeat Rate & Churn Rate (birden fazla alışveriş yapan müşteri sayısı / tüm müşteriler)
# 5. Profit Margin (profit_margin =  total_price * 0.10)
# 6. Customer Value (customer_value = average_order_value * purchase_frequency)
# 7. Customer Lifetime Value (CLTV = (customer_value / churn_rate) x profit_margin)
# 8. Segmentlerin Oluşturulması
# 9. BONUS: Tüm İşlemlerin Fonksiyonlaştırılması

##################################################
# 1. Veri Hazırlama
##################################################

# Veri Seti Hikayesi
# https://archive.ics.uci.edu/ml/datasets/Online+Retail+II

# Online Retail II isimli veri seti İngiltere merkezli online bir satış mağazasının
# 01/12/2009 - 09/12/2011 tarihleri arasındaki satışlarını içeriyor.

# Değişkenler
# InvoiceNo: Fatura numarası. Her işleme yani faturaya ait eşsiz numara. C ile başlıyorsa iptal edilen işlem.
# StockCode: Ürün kodu. Her bir ürün için eşsiz numara.
# Description: Ürün ismi
# Quantity: Ürün adedi. Faturalardaki ürünlerden kaçar tane satıldığını ifade etmektedir.
# InvoiceDate: Fatura tarihi ve zamanı.
# UnitPrice: Ürün fiyatı (Sterlin cinsinden)
# CustomerID: Eşsiz müşteri numarası
# Country: Ülke ismi. Müşterinin yaşadığı ülke.



# ***VERİ HAZIRLAMA***

In [ ]:

from sklearn.preprocessing import MinMaxScaler

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [2]:

df_ = pd.read_excel("/content/drive/MyDrive/Kurs Materyalleri(CRM Analitiği)/datasets/online_retail_II.xlsx", sheet_name="Year 2009-2010")
df = df_.copy()
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95000,13085.00000,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75000,13085.00000,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75000,13085.00000,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10000,13085.00000,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25000,13085.00000,United Kingdom


In [3]:
df.isnull().sum()
df = df[~df["Invoice"].str.contains("C", na=False)]
df.describe().T
df = df[(df['Quantity'] > 0)]
df.dropna(inplace=True)

In [4]:

df["TotalPrice"] = df["Quantity"] * df["Price"]


In [5]:
df

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95000,13085.00000,United Kingdom,83.40000
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75000,13085.00000,United Kingdom,81.00000
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75000,13085.00000,United Kingdom,81.00000
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10000,13085.00000,United Kingdom,100.80000
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25000,13085.00000,United Kingdom,30.00000
...,...,...,...,...,...,...,...,...,...
525456,538171,22271,FELTCRAFT DOLL ROSIE,2,2010-12-09 20:01:00,2.95000,17530.00000,United Kingdom,5.90000
525457,538171,22750,FELTCRAFT PRINCESS LOLA DOLL,1,2010-12-09 20:01:00,3.75000,17530.00000,United Kingdom,3.75000
525458,538171,22751,FELTCRAFT PRINCESS OLIVIA DOLL,1,2010-12-09 20:01:00,3.75000,17530.00000,United Kingdom,3.75000
525459,538171,20970,PINK FLORAL FELTCRAFT SHOULDER BAG,2,2010-12-09 20:01:00,3.75000,17530.00000,United Kingdom,7.50000


In [8]:

cltv_c = df.groupby('Customer ID').agg({'Invoice': lambda x: x.nunique(),
                                        'Quantity': lambda x: x.sum(),
                                        'TotalPrice': lambda x: x.sum()})

cltv_c.columns = ['total_transaction', 'total_unit', 'total_price']

cltv_c

,total_transaction,total_unit,total_price
Customer ID,,,
12346.00000,11,70,372.86000
12347.00000,2,828,1323.32000
12348.00000,1,373,222.16000
12349.00000,3,993,2671.14000
12351.00000,1,261,300.93000
...,...,...,...
18283.00000,6,336,641.77000
18284.00000,1,494,461.68000
18285.00000,1,145,427.00000


# ***AVERAGE ORDER VALUE***

In [9]:
cltv_c.head()
cltv_c["average_order_value"] = cltv_c["total_price"] / cltv_c["total_transaction"]


In [10]:
cltv_c

,total_transaction,total_unit,total_price,average_order_value
Customer ID,,,,
12346.00000,11,70,372.86000,33.89636
12347.00000,2,828,1323.32000,661.66000
12348.00000,1,373,222.16000,222.16000
12349.00000,3,993,2671.14000,890.38000
12351.00000,1,261,300.93000,300.93000
...,...,...,...,...
18283.00000,6,336,641.77000,106.96167
18284.00000,1,494,461.68000,461.68000
18285.00000,1,145,427.00000,427.00000


# ***PURCHASE FREQUENCY***

In [11]:
cltv_c.head()

,total_transaction,total_unit,total_price,average_order_value
Customer ID,,,,
12346.00000,11,70,372.86000,33.89636
12347.00000,2,828,1323.32000,661.66000
12348.00000,1,373,222.16000,222.16000
12349.00000,3,993,2671.14000,890.38000
12351.00000,1,261,300.93000,300.93000


In [12]:
cltv_c['total_transaction'] / cltv_c.shape[0]

,total_transaction
Customer ID,
12346.00000,0.00255
12347.00000,0.00046
12348.00000,0.00023
12349.00000,0.00070
12351.00000,0.00023
...,...
18283.00000,0.00139
18284.00000,0.00023
18285.00000,0.00023


In [14]:
cltv_c["purchase_frequency"] = cltv_c["total_transaction"] / cltv_c.shape[0]

cltv_c.head()

,total_transaction,total_unit,total_price,average_order_value,purchase_frequency
Customer ID,,,,,
12346.00000,11,70,372.86000,33.89636,0.00255
12347.00000,2,828,1323.32000,661.66000,0.00046
12348.00000,1,373,222.16000,222.16000,0.00023
12349.00000,3,993,2671.14000,890.38000,0.00070
12351.00000,1,261,300.93000,300.93000,0.00023


# ***REPEAT RATE & CHURN RATE***

In [16]:

repeat_rate = cltv_c[cltv_c["total_transaction"] > 1].shape[0] / cltv_c.shape[0]
repeat_rate

0.6706073249884098

In [17]:
churn_rate= 1-repeat_rate
churn_rate

0.3293926750115902

# ***PROFIT MARGIN***

In [18]:
cltv_c["profit_margin"]= cltv_c["total_price"]*0.10

In [19]:
cltv_c

,total_transaction,total_unit,total_price,average_order_value,purchase_frequency,profit_margin
Customer ID,,,,,,
12346.00000,11,70,372.86000,33.89636,0.00255,37.28600
12347.00000,2,828,1323.32000,661.66000,0.00046,132.33200
12348.00000,1,373,222.16000,222.16000,0.00023,22.21600
12349.00000,3,993,2671.14000,890.38000,0.00070,267.11400
12351.00000,1,261,300.93000,300.93000,0.00023,30.09300
...,...,...,...,...,...,...
18283.00000,6,336,641.77000,106.96167,0.00139,64.17700
18284.00000,1,494,461.68000,461.68000,0.00023,46.16800
18285.00000,1,145,427.00000,427.00000,0.00023,42.70000


# ***CUSTOMER VALUE***

In [20]:
cltv_c["customer_value"] = cltv_c["average_order_value"] * cltv_c["purchase_frequency"]
cltv_c.head()

,total_transaction,total_unit,total_price,average_order_value,purchase_frequency,profit_margin,customer_value
Customer ID,,,,,,,
12346.00000,11,70,372.86000,33.89636,0.00255,37.28600,0.08643
12347.00000,2,828,1323.32000,661.66000,0.00046,132.33200,0.30675
12348.00000,1,373,222.16000,222.16000,0.00023,22.21600,0.05150
12349.00000,3,993,2671.14000,890.38000,0.00070,267.11400,0.61918
12351.00000,1,261,300.93000,300.93000,0.00023,30.09300,0.06976


# ***CUSTOMER LIFETIME VALUE***

In [ ]:

cltv_c["cltv"] = (cltv_c["customer_value"] / churn_rate) * cltv_c["profit_margin"]

cltv_c.sort_values(by="cltv", ascending=False).head()


In [21]:
cltv_c

,total_transaction,total_unit,total_price,average_order_value,purchase_frequency,profit_margin,customer_value
Customer ID,,,,,,,
12346.00000,11,70,372.86000,33.89636,0.00255,37.28600,0.08643
12347.00000,2,828,1323.32000,661.66000,0.00046,132.33200,0.30675
12348.00000,1,373,222.16000,222.16000,0.00023,22.21600,0.05150
12349.00000,3,993,2671.14000,890.38000,0.00070,267.11400,0.61918
12351.00000,1,261,300.93000,300.93000,0.00023,30.09300,0.06976
...,...,...,...,...,...,...,...
18283.00000,6,336,641.77000,106.96167,0.00139,64.17700,0.14876
18284.00000,1,494,461.68000,461.68000,0.00023,46.16800,0.10702
18285.00000,1,145,427.00000,427.00000,0.00023,42.70000,0.09898


In [23]:
cltv_c["cltv"] = (cltv_c["customer_value"] / churn_rate) * cltv_c["profit_margin"]
cltv_c.sort_values(by="cltv", ascending=False).head()

,total_transaction,total_unit,total_price,average_order_value,purchase_frequency,profit_margin,customer_value,cltv
Customer ID,,,,,,,,
18102.00000,89,124216,349164.35000,3923.19494,0.02063,34916.43500,80.93749,8579573.77276
14646.00000,78,170342,248396.50000,3184.57051,0.01808,24839.65000,57.57916,4342070.45829
14156.00000,102,108107,196566.74000,1927.12490,0.02364,19656.67400,45.56484,2719105.08615
14911.00000,205,69722,152147.57000,742.18327,0.04752,15214.75700,35.26833,1629055.80978
13694.00000,94,125893,131443.19000,1398.33181,0.02179,13144.31900,30.46898,1215855.89003


In [24]:
cltv_c.describe().T

,count,mean,std,min,25%,50%,75%,max
total_transaction,4314.00000,4.45410,8.16866,1.00000,1.00000,2.00000,5.00000,205.00000
total_unit,4314.00000,1284.01113,6458.45205,1.00000,158.00000,382.00000,995.25000,220600.00000
total_price,4314.00000,2047.28866,8912.52324,0.00000,307.95000,705.55000,1722.80250,349164.35000
average_order_value,4314.00000,378.14723,492.51721,0.00000,181.95632,286.94600,423.53375,11880.84000
purchase_frequency,4314.00000,0.00103,0.00189,0.00023,0.00023,0.00046,0.00116,0.04752
profit_margin,4314.00000,204.72887,891.25232,0.00000,30.79500,70.55500,172.28025,34916.43500
customer_value,4314.00000,0.47457,2.06595,0.00000,0.07138,0.16355,0.39935,80.93749
cltv,4314.00000,5883.60651,156068.63594,0.00000,6.67369,35.03173,208.87042,8579573.77276


# ***CREATING SEGMENTS***

In [25]:
cltv_c.sort_values(by="cltv", ascending=False).tail()

,total_transaction,total_unit,total_price,average_order_value,purchase_frequency,profit_margin,customer_value,cltv
Customer ID,,,,,,,,
15913.00000,1,3,6.30000,6.30000,0.00023,0.63000,0.00146,0.00279
13788.00000,1,1,3.75000,3.75000,0.00023,0.37500,0.00087,0.00099
14095.00000,1,1,2.95000,2.95000,0.00023,0.29500,0.00068,0.00061
14103.00000,1,5,0.00000,0.00000,0.00023,0.00000,0.00000,0.00000
14827.00000,1,5,0.00000,0.00000,0.00023,0.00000,0.00000,0.00000


In [26]:
cltv_c.sort_values(by="cltv", ascending=False).head()

,total_transaction,total_unit,total_price,average_order_value,purchase_frequency,profit_margin,customer_value,cltv
Customer ID,,,,,,,,
18102.00000,89,124216,349164.35000,3923.19494,0.02063,34916.43500,80.93749,8579573.77276
14646.00000,78,170342,248396.50000,3184.57051,0.01808,24839.65000,57.57916,4342070.45829
14156.00000,102,108107,196566.74000,1927.12490,0.02364,19656.67400,45.56484,2719105.08615
14911.00000,205,69722,152147.57000,742.18327,0.04752,15214.75700,35.26833,1629055.80978
13694.00000,94,125893,131443.19000,1398.33181,0.02179,13144.31900,30.46898,1215855.89003


In [27]:
cltv_c["segment"] = pd.qcut(cltv_c["cltv"], 4, labels=["D", "C", "B", "A"])

In [28]:
cltv_c

,total_transaction,total_unit,total_price,average_order_value,purchase_frequency,profit_margin,customer_value,cltv,segment
Customer ID,,,,,,,,,
12346.00000,11,70,372.86000,33.89636,0.00255,37.28600,0.08643,9.78357,C
12347.00000,2,828,1323.32000,661.66000,0.00046,132.33200,0.30675,123.23546,B
12348.00000,1,373,222.16000,222.16000,0.00023,22.21600,0.05150,3.47326,D
12349.00000,3,993,2671.14000,890.38000,0.00070,267.11400,0.61918,502.11041,A
12351.00000,1,261,300.93000,300.93000,0.00023,30.09300,0.06976,6.37290,D
...,...,...,...,...,...,...,...,...,...
18283.00000,6,336,641.77000,106.96167,0.00139,64.17700,0.14876,28.98443,C
18284.00000,1,494,461.68000,461.68000,0.00023,46.16800,0.10702,14.99989,C
18285.00000,1,145,427.00000,427.00000,0.00023,42.70000,0.09898,12.83103,C


In [29]:
cltv_c.sort_values(by="cltv", ascending=False).tail()

,total_transaction,total_unit,total_price,average_order_value,purchase_frequency,profit_margin,customer_value,cltv,segment
Customer ID,,,,,,,,,
15913.00000,1,3,6.30000,6.30000,0.00023,0.63000,0.00146,0.00279,D
13788.00000,1,1,3.75000,3.75000,0.00023,0.37500,0.00087,0.00099,D
14095.00000,1,1,2.95000,2.95000,0.00023,0.29500,0.00068,0.00061,D
14103.00000,1,5,0.00000,0.00000,0.00023,0.00000,0.00000,0.00000,D
14827.00000,1,5,0.00000,0.00000,0.00023,0.00000,0.00000,0.00000,D


In [30]:
cltv_c.groupby("segment").agg({"count", "mean", "sum"})

<ipython-input-30-ed5355db4bdb>:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  cltv_c.groupby("segment").agg({"count", "mean", "sum"})


total_transaction              total_unit                total_price  \
                     mean    sum count       mean      sum count        mean   
segment                                                                        
D                 1.22892   1326  1079  109.00463   117616  1079   178.18826   
C                 2.00371   2160  1078  283.05659   305135  1078   475.89652   
B                 3.76902   4063  1078  680.15863   733211  1078  1131.35918   
A                10.81186  11666  1079 4062.33735  4383262  1079  6401.40546   

                            average_order_value                     \
                  sum count                mean          sum count   
segment                                                              
D        192265.13000  1079           157.37300 169805.46550  1079   
C        513016.45300  1078           294.45487 317422.35514  1078   
B       1219605.20000  1078           389.72832 420127.12365  1078   
A       6907116.49100  1079           670.96592 723972.22347  1079   

        purchase_frequency               profit_margin                     \
                      mean     sum count          mean          sum count   
segment                                                                     
D                  0.00028 0.30737  1079      17.81883  19226.51300  1079   
C                  0.00046 0.50070  1078      47.58965  51301.64530  1078   
B                  0.00087 0.94182  1078     113.13592 121960.52000  1078   
A                  0.00251 2.70422  1079     640.14055 690711.64910  1079   

        customer_value                         cltv                       
                  mean        sum count        mean            sum count  
segment                                                                   
D              0.04130   44.56772  1079     2.64071     2849.32790  1079  
C              0.11031  118.91897  1078    16.86852    18184.26342  1078  
B              0.26225  282.70867  1078    96.05739   103549.86329  1078  
A              1.48387 1601.09330  1079 23408.05843 25257295.04118  1079

In [31]:
cltv_c.to_csv("cltc_c.csv")

# ***FUNCTIONALIZATION***

In [32]:
def create_cltv_c(dataframe, profit=0.10):

    # Veriyi hazırlama
    dataframe = dataframe[~dataframe["Invoice"].str.contains("C", na=False)]
    dataframe = dataframe[(dataframe['Quantity'] > 0)]
    dataframe.dropna(inplace=True)
    dataframe["TotalPrice"] = dataframe["Quantity"] * dataframe["Price"]
    cltv_c = dataframe.groupby('Customer ID').agg({'Invoice': lambda x: x.nunique(),
                                                   'Quantity': lambda x: x.sum(),
                                                   'TotalPrice': lambda x: x.sum()})
    cltv_c.columns = ['total_transaction', 'total_unit', 'total_price']
    # avg_order_value
    cltv_c['avg_order_value'] = cltv_c['total_price'] / cltv_c['total_transaction']
    # purchase_frequency
    cltv_c["purchase_frequency"] = cltv_c['total_transaction'] / cltv_c.shape[0]
    # repeat rate & churn rate
    repeat_rate = cltv_c[cltv_c.total_transaction > 1].shape[0] / cltv_c.shape[0]
    churn_rate = 1 - repeat_rate
    # profit_margin
    cltv_c['profit_margin'] = cltv_c['total_price'] * profit
    # Customer Value
    cltv_c['customer_value'] = (cltv_c['avg_order_value'] * cltv_c["purchase_frequency"])
    # Customer Lifetime Value
    cltv_c['cltv'] = (cltv_c['customer_value'] / churn_rate) * cltv_c['profit_margin']
    # Segment
    cltv_c["segment"] = pd.qcut(cltv_c["cltv"], 4, labels=["D", "C", "B", "A"])

    return cltv_c





In [33]:
df = df_.copy()

clv = create_cltv_c(df)

In [34]:
df

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95000,13085.00000,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75000,13085.00000,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75000,13085.00000,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10000,13085.00000,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25000,13085.00000,United Kingdom
...,...,...,...,...,...,...,...,...
525456,538171,22271,FELTCRAFT DOLL ROSIE,2,2010-12-09 20:01:00,2.95000,17530.00000,United Kingdom
525457,538171,22750,FELTCRAFT PRINCESS LOLA DOLL,1,2010-12-09 20:01:00,3.75000,17530.00000,United Kingdom
525458,538171,22751,FELTCRAFT PRINCESS OLIVIA DOLL,1,2010-12-09 20:01:00,3.75000,17530.00000,United Kingdom
525459,538171,20970,PINK FLORAL FELTCRAFT SHOULDER BAG,2,2010-12-09 20:01:00,3.75000,17530.00000,United Kingdom


In [35]:
clv

,total_transaction,total_unit,total_price,avg_order_value,purchase_frequency,profit_margin,customer_value,cltv,segment
Customer ID,,,,,,,,,
12346.00000,11,70,372.86000,33.89636,0.00255,37.28600,0.08643,9.78357,C
12347.00000,2,828,1323.32000,661.66000,0.00046,132.33200,0.30675,123.23546,B
12348.00000,1,373,222.16000,222.16000,0.00023,22.21600,0.05150,3.47326,D
12349.00000,3,993,2671.14000,890.38000,0.00070,267.11400,0.61918,502.11041,A
12351.00000,1,261,300.93000,300.93000,0.00023,30.09300,0.06976,6.37290,D
...,...,...,...,...,...,...,...,...,...
18283.00000,6,336,641.77000,106.96167,0.00139,64.17700,0.14876,28.98443,C
18284.00000,1,494,461.68000,461.68000,0.00023,46.16800,0.10702,14.99989,C
18285.00000,1,145,427.00000,427.00000,0.00023,42.70000,0.09898,12.83103,C
